In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [4]:
!kaggle datasets download -d princelv84/dogsvscats

Dataset URL: https://www.kaggle.com/datasets/princelv84/dogsvscats
License(s): MIT
 95% 519M/545M [00:01<00:00, 333MB/s]
100% 545M/545M [00:03<00:00, 172MB/s]


In [5]:
import zipfile
zip_ref=zipfile.ZipFile('/content/dogsvscats.zip','r')
zip_ref.extractall('/content')
zip_ref.close()

In [34]:
import os
import cv2
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,classification_report
from sklearn.preprocessing import StandardScaler

In [35]:
def load_images(folder,label,img_size=64):
  data=[]
  for img_name in os.listdir(folder):
    img_path=os.path.join(folder,img_name)
    img=cv2.imread(img_path)
    if img is not None:
      img=cv2.resize(img,(img_size,img_size))
      img=img.flatten()
      data.append([img,label])
    return data

In [36]:
train_data=[]
train_data+=load_images("train/cats",0)
train_data+=load_images("train/dogs",1)
np.random.shuffle(train_data)
x_train=np.array([i[0] for i in train_data])
y_train=np.array([i[1] for i in train_data])


In [37]:
test_data=[]
test_data+=load_images("test/cats",0)
test_data+=load_images("test/dogs",1)
x_test=np.array([i[0] for i in test_data])
y_test=np.array([i[1] for i in test_data])

In [38]:
scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)

In [39]:
svm=SVC(kernel='linear',C=1,gamma='scale')
svm.fit(x_train,y_train)

SVC(C=1, kernel='linear')

In [40]:
y_pred=svm.predict(x_test)

In [41]:
print("Accuracy:",accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

Accuracy: 0.5
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.50      1.00      0.67         1

    accuracy                           0.50         2
   macro avg       0.25      0.50      0.33         2
weighted avg       0.25      0.50      0.33         2



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [44]:
def predict_image(img_path, model,img_size=64):
    img = cv2.imread(img_path)
    if img is None:
      print("Image not found")
      return
    img = cv2.resize(img, (img_size, img_size))
    img = img.flatten()
    img = img / 255.0

    pred = model.predict([img])[0]

    if pred == 0:
        print("Prediction: Cat")
    else:
        print("Prediction:Dog")

In [48]:
predict_image("test/cats/cat.10207.jpg",svm)

Prediction: Cat
